# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from Config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
City_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
City_df.head()

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Nikolskoye,59.7035,30.7861,62.49,84,42,3.29,RU,2022-08-03 23:12:05
1,Ushuaia,-54.8000,-68.3000,37.06,87,40,3.44,AR,2022-08-03 23:11:36
2,Tasiilaq,65.6145,-37.6368,33.96,71,79,3.15,GL,2022-08-03 23:12:48
3,Charter Township of Clinton,42.5870,-82.9199,75.22,97,100,10.36,US,2022-08-03 23:16:29
4,Coahuayana Viejo,18.7333,-103.6833,88.52,63,97,5.88,MX,2022-08-03 23:19:56


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#config Key
gmaps.configure(api_key=g_key)

In [4]:
#Using the Lat and Lng as locations and Humidity as weight.
locations = City_df[["Lat", "Lng"]]
weights = City_df["Humidity (%)"]

figure_layout = {
    'width': "100%",
    'height': '400px',
    'padding': '2px',
}

ActualMap = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
ActualMap.add_layer(heatmap_layer)

heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3
ActualMap

Figure(layout=FigureLayout(height='400px', padding='2px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition.

#For example:
#A max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph
#Zero cloudiness
ideal_destinations = City_df.loc[(City_df["Max Temp (F)"] > 70) & 
                               (City_df["Max Temp (F)"] < 80) & 
                               (City_df["Wind Speed (mph)"] < 10) & 
                               (City_df["Cloudiness (%)"] == 0), :].reset_index(drop=True).dropna()
ideal_destinations

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Saint-François,46.4154,3.9054,71.04,64,0,2.30,FR,2022-08-03 23:18:50
1,Alexandria,31.2156,29.9553,78.75,68,0,9.19,EG,2022-08-03 23:17:37
2,Cabra,37.4725,-4.4421,78.01,36,0,4.61,ES,2022-08-03 23:21:17
3,Jilin,43.0000,126.0000,74.46,91,0,4.79,CN,2022-08-03 23:22:31
4,Vieste,41.8825,16.1750,78.67,62,0,9.60,IT,2022-08-03 23:22:40
5,Neyshabur,36.2133,58.7958,75.20,28,0,2.84,IR,2022-08-03 23:22:43
6,Daşoguz,41.8363,59.9666,73.35,49,0,6.91,TM,2022-08-03 23:22:56
7,Belmonte,-15.8631,-38.8828,70.43,76,0,7.20,BR,2022-08-03 23:23:13
8,Province of Mantua,45.1667,10.7833,79.84,72,0,1.99,IT,2022-08-03 23:23:30
9,Akdepe,42.0551,59.3788,73.49,35,0,5.57,TM,2022-08-03 23:23:48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_destinations[["City", "Country", "Lat", "Lng"]].copy()

#Add a "Hotel Name" column to the DataFrame with null values
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Saint-François,FR,46.4154,3.9054,NaN
1,Alexandria,EG,31.2156,29.9553,NaN
2,Cabra,ES,37.4725,-4.4421,NaN
3,Jilin,CN,43.0000,126.0000,NaN
4,Vieste,IT,41.8825,16.1750,NaN
5,Neyshabur,IR,36.2133,58.7958,NaN
6,Daşoguz,TM,41.8363,59.9666,NaN
7,Belmonte,BR,-15.8631,-38.8828,NaN
8,Province of Mantua,IT,45.1667,10.7833,NaN
9,Akdepe,TM,42.0551,59.3788,NaN


In [7]:
#Set up parameters for querying Google Places API
target_search = "hotel"
target_radius = 5000
target_type = "lodging"


Parameters_Dict = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

#define base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
#For loop set up to go through each city in the dataframe and make an API call

for index, row in hotel_df.iterrows():
    
    
    lat = row["Lat"]
    lng = row["Lng"]
    Parameters_Dict["location"] = f"{lat},{lng}"
    

    hotel_data = requests.get(base_url, Parameters_Dict).json()
    

    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' is a match!!! The nearest hotel is {hotel_name}")
        print("**************************************************************************")
              
    except:
        print(f"Could not find information for '{row['City']}'... moving on to next potential city on the list.")    
        print("**************************************************************************")

City 'Saint-François' is a match!!! The nearest hotel is Maison Vin d'Hus
**************************************************************************
City 'Alexandria' is a match!!! The nearest hotel is Hilton Alexandria Corniche
**************************************************************************
City 'Cabra' is a match!!! The nearest hotel is MS Fuente Las Piedras
**************************************************************************
Could not find information for 'Jilin'... moving on to next potential city on the list.
**************************************************************************
City 'Vieste' is a match!!! The nearest hotel is Pizzomunno Vieste Palace Hotel
**************************************************************************
City 'Neyshabur' is a match!!! The nearest hotel is Eram Hotel
**************************************************************************
City 'Daşoguz' is a match!!! The nearest hotel is Dashouz Hotel
*******************************

In [9]:
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Saint-François,FR,46.4154,3.9054,Maison Vin d'Hus
1,Alexandria,EG,31.2156,29.9553,Hilton Alexandria Corniche
2,Cabra,ES,37.4725,-4.4421,MS Fuente Las Piedras
3,Vieste,IT,41.8825,16.1750,Pizzomunno Vieste Palace Hotel
4,Neyshabur,IR,36.2133,58.7958,Eram Hotel
5,Daşoguz,TM,41.8363,59.9666,Dashouz Hotel
6,Belmonte,BR,-15.8631,-38.8828,Pousada do Diogo
7,Province of Mantua,IT,45.1667,10.7833,Hotel la Favorita
8,Sudak,UA,44.8500,34.9667,Sota


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
ActualMap.add_layer(markers)

# Display interactive map
ActualMap

Figure(layout=FigureLayout(height='400px', padding='2px', width='100%'))